In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd

### 제주도 농어촌민박(펜션, 게스트하우스, 민박 등)
- 출처 - 제주데이터허브
- https://jejudatahub.net/data/view/data/956


In [2]:
df_accomo = pd.read_csv('./data/제주농어촌민박현황.csv')
df_accomo.head(20)

,읍면동,민박명,소재지,데이터기준일
0,한림읍,기영 민박,제주시 한림읍 한림로 345,20211123
1,한림읍,에너벨리,제주시 한림읍 한림로 356,20211123
2,한림읍,협재 빌리지,제주시 한림읍 한림로 338-1,20211123
3,한림읍,해송민박,제주시 한림읍 협재4길 7-15,20211123
4,한림읍,영심 민박,제주시 한림읍 협재4길 7-7,20211123
5,한림읍,원룸민박,제주시 한림읍 비양도길 275,20211123
6,한림읍,바다그리기,제주시 한림읍 옹포2길 16,20211123
7,한림읍,비양도 민박,제주시 한림읍 비양도길 8,20211123
8,한림읍,해조대 민박,제주시 한림읍 한림로 339,20211123
9,한림읍,이쁜새민박,제주시 한림읍 협재로 68-5,20211123


In [3]:
accomo_list = df_accomo['민박명']
accomo_list

0              기영 민박
1               에너벨리
2             협재 빌리지
3               해송민박
4              영심 민박
            ...     
2979         화랑펜션2층집
2980    김군빌리지 게스트하우스
2981           도두봉팬션
2982           바다성펜션
2983             올레뷰
Name: 민박명, Length: 2984, dtype: object

### 네이버 블로그 데이터 확인



In [4]:
# 제주 + "제주낭만펜션" 검색결과 URL
# "" 포함시 필수검색 옵션
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+%22%EB%82%AD%EB%A7%8C%EC%A0%9C%EC%A3%BC%ED%8E%9C%EC%85%98%22+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C&tqi'

In [21]:
base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
accomo_name = quote(f'"{accomo_list[10]}"')

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_search = '+-%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C+-%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + accomo_name + delete_search

driver = webdriver.Chrome()
driver.get(url)

In [22]:
quote('파트너스')

'%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4'

In [ ]:
base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
accomo_name = quote(f'"{accomo_list[10]}"')

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_search = f'+-{quote("파트너스")}+-{quote("예약코드")}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + accomo_name + delete_search

driver = webdriver.Chrome()
driver.get(url)

In [23]:
delete_words = ["파트너스", "예약코드"]
delete_words

['파트너스', '예약코드']

In [24]:
quote(f'"{delete_words[0]}"')

'%22%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4%22'

In [ ]:
base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
accomo_name = quote(f'"{accomo_list[3]}"')

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_words = ["파트너스", "예약코드"]
delete_words

delete_search = ""
for word in delete_words:    
    delete_search += f'+-{quote(word)}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + accomo_name + delete_search

driver = webdriver.Chrome()
driver.get(url)

In [10]:
base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
accomo_name = quote(f'"{accomo_list[3]}"')

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_words = ["파트너스", "예약코드"]
delete_words

delete_search = ""
for word in delete_words:    
    delete_search += f'+-{quote(word)}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + accomo_name + delete_search

driver = webdriver.Chrome()
driver.get(url)
driver.execute_script('window.scrollTo(0, 5500)')


# 스크롤 내리기
for i in range(10):
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)    
    time.sleep(round(random.uniform(0.0, 1.2), 3))
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break




In [43]:
base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
accomo_name = quote(f'"{accomo_list[5]}"')

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_words = ["파트너스", "예약코드"]
delete_words

delete_search = ""
for word in delete_words:    
    delete_search += f'+-{quote(word)}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + accomo_name + delete_search

driver = webdriver.Chrome()
driver.get(url)


# 스크롤 내리기
for _ in range(3):
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(0.0, 1.2), 3))

    # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select_one('.lst_view').select('.view_wrap')

blog = []

for li in lis:
    name = accomo_name
    title = li.select_one('.title_area').text.strip()
    blog_link = li.select_one('.title_area').select_one('a')['href']
    blog.append({'민박' : name,
                 'title' : title,
                 'link' : blog_link})
    

    


In [44]:
pd.DataFrame(blog)


,민박,title,link
0,예약코드,2023 제주 서부 여행 16일차 [숙소 주월담 후기],https://blog.naver.com/flute1017/223183879709
1,예약코드,서귀포 안덕면 제주할망밥상 제주화순항 맛집 백반맛집...,https://blog.naver.com/pigrabbitnana/223204879419
2,예약코드,비양도(제주),https://blog.naver.com/sks6012/223288837255
3,예약코드,제주안덕면밥집 제주할망밥상 화순점,https://blog.naver.com/sjehyn/222930637481
4,예약코드,"제주도 워케이션 제주감성숙소, 독채펜션 제주숙박 으로 힐링하자",https://blog.naver.com/spxl1010/222925235440
...,...,...,...
115,예약코드,제주도 Day-6,https://blog.naver.com/sjjang61/88092659
116,예약코드,일본 : 파란만장 오사카 여행①,https://blog.naver.com/lee_mo_ri/220752069925
117,예약코드,임야담보대출 좋은조건 75까지로,http://thegreenfarm.kr/9661856137
118,예약코드,[제주도 대지] 조천읍 함덕리 준 주거지역에 위치하고...,https://blog.naver.com/gt9969/221935773740


여기서 부터

In [33]:
result = []
cnt = 0

for accomo in tqdm(accomo_list[:10]):

    # 숙소 이름별 - 검색
    base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query=%EC%A0%9C%EC%A3%BC+'
    accomo_name = quote(f'"{accomo}"')

    # 제외하고자 하는 검색어
    # 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
    # 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
    delete_words = ["파트너스", "예약코드"]
    delete_words

    delete_search = ""
    for word in delete_words:    
        delete_search += f'+-{quote(word)}'

    # 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
    url = base_url + accomo_name + delete_search
    driver = webdriver.Chrome()
    driver.get(url)


    # 스크롤 내리기

    # len(soup.select('.detail_box')) = 30 첫페이지에 나오는 수가 30개
    # 30개 이하인 경우 분석에서 제외
    if len(driver.find_elements(By.CLASS_NAME, 'detail_box')) > 30:
        continue
    
    else :
        # 스크롤은 3번만 내리자
        for _ in range(3):
            last_height = driver.execute_script("return document.body.scrollHeight")

            # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
            driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
            time.sleep(round(random.uniform(0.0, 1.0), 3))
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
            time.sleep(round(random.uniform(0.0, 1.2), 3))

            # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
    
    time.sleep(round(random.uniform(0.0, 2.2), 3))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select_one('.lst_view').select('.view_wrap')
    
    title_list = []
    content_list = []
    blog_list = []
    time_list = []

    # 블로그가 10개 미만인 경우 분석에서 제외
    if len(lis) >= 10:
        for li in lis:
            title_raw = li.select_one('.title_area').select_one('a').text.strip()
            #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
            title_list.append(title_raw)

            content_raw = li.select_one('.dsc_link').text.strip()
            #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
            content_list.append(content_raw)  
                      
            blog_link = li.select_one('.title_area').select_one('a')['href']
            blog_list.append(blog_link)

            blog_time = li.select_one('.user_info').select_one('span').text.strip()
            time_list.append(blog_time)

    result.append({'name' : accomo,
                   'title' : title_list,
                   'contents' : content_list,
                   'blog' : blog_list,
                   'time' : time_list})
    cnt += 1
    time.sleep(round(random.uniform(0.0, 1.2), 3))

    print(f'{cnt}번째 "{accomo}" 블로그 링크 수집완료')
time.sleep(round(random.uniform(0.0, 4.2), 3))
        


 10%|█         | 1/10 [00:06<00:56,  6.22s/it]

1번째 "기영 민박" 블로그 링크 수집완료


 10%|█         | 1/10 [00:11<01:47, 11.89s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.200)
Stacktrace:
	GetHandleVerifier [0x00007FF7F50B2142+3514994]
	(No symbol) [0x00007FF7F4CD0CE2]
	(No symbol) [0x00007FF7F4B776AA]
	(No symbol) [0x00007FF7F4B50AFD]
	(No symbol) [0x00007FF7F4BECB1B]
	(No symbol) [0x00007FF7F4C0218F]
	(No symbol) [0x00007FF7F4BE5D93]
	(No symbol) [0x00007FF7F4BB4BDC]
	(No symbol) [0x00007FF7F4BB5C64]
	GetHandleVerifier [0x00007FF7F50DE16B+3695259]
	GetHandleVerifier [0x00007FF7F5136737+4057191]
	GetHandleVerifier [0x00007FF7F512E4E3+4023827]
	GetHandleVerifier [0x00007FF7F4E004F9+689705]
	(No symbol) [0x00007FF7F4CDC048]
	(No symbol) [0x00007FF7F4CD8044]
	(No symbol) [0x00007FF7F4CD81C9]
	(No symbol) [0x00007FF7F4CC88C4]
	BaseThreadInitThunk [0x00007FFD95A47344+20]
	RtlUserThreadStart [0x00007FFD95B826B1+33]


In [34]:
result = pd.DataFrame(result)
result['blog_num'] = result['blog'].apply(lambda x: len(x))
result

,name,title,contents,blog,time,blog_num
0,기영 민박,"[[제주여행] 제주 마지막날 Day3 / 바다에서 하염없이 멍때리기..., 제주 혼...",[옮겨 #기영민박 들려서 맥주 병나발 불고 육지로 돌아가기 싫음을 온몸으로 표현 돈...,"[https://blog.naver.com/imvaney/223121121422, ...","[2023.06.05., 2023.11.26., 2016.12.13., 2021.0...",90


In [18]:
blog_url = 'https://blog.naver.com/flute1017/223183879709'
driver = webdriver.Chrome()
driver.get(blog_url)
driver.switch_to.frame('mainFrame')


lis = driver.find_element(By.CLASS_NAME, 'se-main-container').find_elements(By.CLASS_NAME, 'se-text')
contents = []

for li in lis:
    content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())
    contents.append(content.replace('\U0001f979','').replace('\n',''))
    text = ' '.join(contents)

In [54]:
for idx, ser in df.iterrows():
    name = ser['name']
    print(name)
    cnt = 0
    for blog_link in ser['blog']:        
        # naver 블로그만 선택
        if blog_link.find('blog.naver.com') > 0:
            print(blog_link)
            cnt += 1
            if cnt > 30:
                break
        

기영 민박
https://blog.naver.com/imvaney/223121121422
https://blog.naver.com/josiesbottleshop/223275216016
https://blog.naver.com/youngsugi471/220884814752
https://blog.naver.com/iamkimin/222195563764
https://blog.naver.com/nomadlife80/222419017686
https://blog.naver.com/iam-6/220583052225
https://blog.naver.com/jmh4047/221176159709
https://blog.naver.com/pious86/220491355835
https://blog.naver.com/youngsugi471/220798234189
https://blog.naver.com/youngsugi471/220792952169
https://blog.naver.com/youngsugi471/220403012299
https://blog.naver.com/youngsugi471/220321533428
https://blog.naver.com/youngsugi471/220798118157
https://blog.naver.com/youngsugi471/220888972624
https://blog.naver.com/youngsugi471/220437866825
https://blog.naver.com/youngsugi471/221088527170
https://blog.naver.com/youngsugi471/221719914120
https://blog.naver.com/youngsugi471/220712652034
https://blog.naver.com/youngsugi471/220786303213
https://blog.naver.com/youngsugi471/220592380297
https://blog.naver.com/youngsugi471/2

In [35]:
df =result
df.blog

df1 = df[df['name'] == '기영 민박']
df1

,name,title,contents,blog,time,blog_num
0,기영 민박,"[[제주여행] 제주 마지막날 Day3 / 바다에서 하염없이 멍때리기..., 제주 혼...",[옮겨 #기영민박 들려서 맥주 병나발 불고 육지로 돌아가기 싫음을 온몸으로 표현 돈...,"[https://blog.naver.com/imvaney/223121121422, ...","[2023.06.05., 2023.11.26., 2016.12.13., 2021.0...",90


In [36]:
df1.to_csv('test.csv')

In [37]:
df2 = pd.read_csv('test.csv')
df2

,Unnamed: 0,name,title,contents,blog,time,blog_num
0,0,기영 민박,"['[제주여행] 제주 마지막날 Day3 / 바다에서 하염없이 멍때리기...', '제...",['옮겨 #기영민박 들려서 맥주 병나발 불고 육지로 돌아가기 싫음을 온몸으로 표현 ...,['https://blog.naver.com/imvaney/223121121422'...,"['2023.06.05.', '2023.11.26.', '2016.12.13.', ...",90


In [41]:
def convert_date(s):
    if len(s) == 11 : # 2023.12.13.  
        return s[:-1] # 마지막 점 제거
    
    now = datetime.now()
    date = s[:-1]
    if '어제' in s:
        date = now - timedelta(days = 1)
    elif '일' in s:
        date = now - timedelta(days = int(s[0]))
    elif '시간' in s:
        date = now - timedelta(hours = int(s[:-4]))
    elif '분' in s:
        date = now - timedelta(minutes = int(s[:-3]))
    elif '주' in s:
        date = now - timedelta(weeks = int(s[:-3]))

    return date.strftime('%Y.%m.%d')

In [43]:
now = datetime.now()
df2['time'].apply(lambda x: now - timedelta(days=6))

0   2023-12-29 21:14:12.131562
Name: time, dtype: datetime64[ns]

In [69]:
accomo_contents = []
for idx, ser in df1.iterrows():
    name = ser['name']
    print(name)
    cnt = 0
    
    


    for blog_url in tqdm(ser['blog']):        
        # naver 블로그만 선택
        if blog_url.find('blog.naver.com') > 0:
                      
            driver = webdriver.Chrome()
            driver.get(blog_url)
            driver.switch_to.frame('mainFrame')

            try : 
                lis = driver.find_element(By.CLASS_NAME, 'se-main-container').find_elements(By.CLASS_NAME, 'se-text')
                blog_contents = []

                for li in lis:
                    content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())
                    blog_contents.append(content.replace('\U0001f979','').replace('\n',''))
                    text = ' '.join(blog_contents)

                cnt += 1
                accomo_contents.append(blog_contents)
                
            except : 
                try :
                    lis = driver.find_element(By.CLASS_NAME, 'postViewArea').find_elements(By.CLASS_NAME, 'se-text')
                    blog_contents = []

                    for li in lis:
                        content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())
                        blog_contents.append(content.replace('\U0001f979','').replace('\n',''))
                        text = ' '.join(blog_contents)

                    cnt += 1
                    accomo_contents.append(blog_contents)
                     
                except :
                     continue

            if cnt > 30:
                    break
        

기영 민박


100%|██████████| 90/90 [07:16<00:00,  4.85s/it]


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd



In [2]:
df2 = pd.read_csv('test.csv')
df2

,Unnamed: 0,name,title,contents,blog,time,blog_num
0,0,기영 민박,"['[제주여행] 제주 마지막날 Day3 / 바다에서 하염없이 멍때리기...', '제...",['옮겨 #기영민박 들려서 맥주 병나발 불고 육지로 돌아가기 싫음을 온몸으로 표현 ...,['https://blog.naver.com/imvaney/223121121422'...,"['2023.06.05.', '2023.11.26.', '2016.12.13.', ...",90


In [3]:
for i in df2['blog']:
    print(i)

['https://blog.naver.com/imvaney/223121121422', 'https://blog.naver.com/josiesbottleshop/223275216016', 'https://blog.naver.com/youngsugi471/220884814752', 'https://blog.naver.com/iamkimin/222195563764', 'https://blog.naver.com/nomadlife80/222419017686', 'https://blog.naver.com/iam-6/220583052225', 'https://blog.naver.com/jmh4047/221176159709', 'https://blog.naver.com/pious86/220491355835', 'https://blog.naver.com/youngsugi471/220798234189', 'https://blog.naver.com/youngsugi471/220792952169', 'https://blog.naver.com/youngsugi471/220403012299', 'https://blog.naver.com/youngsugi471/220321533428', 'https://blog.naver.com/youngsugi471/220798118157', 'https://blog.naver.com/youngsugi471/220888972624', 'https://blog.naver.com/youngsugi471/220437866825', 'https://blog.naver.com/youngsugi471/221088527170', 'https://blog.naver.com/youngsugi471/221719914120', 'https://blog.naver.com/youngsugi471/220712652034', 'https://blog.naver.com/youngsugi471/220786303213', 'https://blog.naver.com/youngsugi4

In [5]:
df2['time'].unique()

array(["['2023.06.05.', '2023.11.26.', '2016.12.13.', '2021.01.04.', '2021.07.03.', '2015.12.30.', '2018.01.02.', '2015.09.24.', '2016.08.27.', '2016.08.21.', '2015.06.27.', '2015.04.05.', '2016.08.27.', '2016.12.18.', '2015.08.01.', '2017.09.03.', '2019.11.27.', '2016.05.18.', '2016.08.12.', '2016.01.09.', '2016.10.19.', '2016.06.06.', '2016.05.27.', '2016.09.02.', '2016.09.28.', '2016.06.19.', '2015.08.15.', '2016.04.25.', '2014.09.14.', '2015.01.06.', '2016.07.22.', '2014.06.04.', '2016.04.15.', '2016.09.24.', '2023.06.22.', '2015.08.14.', '2017.10.01.', '2016.04.25.', '2016.01.25.', '2016.08.10.', '2016.01.18.', '2016.07.22.', '2016.09.02.', '2015.11.03.', '2016.06.02.', '2016.06.02.', '2021.01.05.', '2017.06.06.', '2015.08.11.', '2015.08.16.', '2015.08.16.', '2015.08.03.', '2015.09.11.', '2015.09.11.', '2015.08.01.', '2015.09.21.', '2016.01.26.', '2016.01.26.', '2018.07.23.', '2015.07.31.', '2016.07.22.', '2014.06.04.', '2016.04.15.', '2016.09.24.', '2023.06.22.', '2015.08.14.', '

In [74]:
# 날짜 변환 함수 생성
def convert_date(s):
    if len(s) == 11 : # 2023.12.13.  
        return s[:-1] # 마지막 점 제거
  
    now = datetime.now()
    if '어제' in s:
        blog_time = now - timedelta(days = 1)
    elif '일' in s:
        blog_time = now - timedelta(days = int(s[0]))
    elif '시간' in s:
        blog_time = now - timedelta(hours = int(s[:-4]))
    elif '분' in s:
        blog_time = now - timedelta(minutes = int(s[:-3]))
    elif '주' in s:
        blog_time = now - timedelta(weeks = int(s[:-3]))

    return blog_time.strftime('%Y.%m.%d')

# date = now - timedelta(weeks=int(re.sub(r'[^0-9]',"",s)))

In [47]:
temp = df2['time'][0]
temp2 = ''.join(temp).split(',')[1]
temp3 = re.sub(r'[^0-9]',"",temp2)
temp3


'20231126'

In [41]:
print(type(temp))

<class 'str'>


In [88]:
for idx, ser in df2.iterrows():
    name = ser['name']
    print(name)
    cnt = 0
    for blog_time in ''.join(ser['time']).split(','):        
        # naver 블로그만 선택
        #print())
        blog_time = re.sub("[\[\]''']","",blog_time.strip())
        blog_time = datetime.strptime(blog_time, '%Y.%m.%d.') 


        now = datetime.now()
        if '어제' in blog_time:
            blog_time = now - timedelta(days = 1)
        elif '일' in blog_time:
            blog_time = now - timedelta(days = int(blog_time[0]))
        elif '시간' in blog_time:
            blog_time = now - timedelta(hours = int(blog_time[:-4]))
        elif '분' in blog_time:
            blog_time = now - timedelta(minutes = int(blog_time[:-3]))
        elif '주' in blog_time:
            blog_time = now - timedelta(weeks = int(blog_time[:-3]))
        
        print(blog_time)
        print(datetime.strptime(blog_time, '%Y.%m.%d.') )
        




           

           

기영 민박
2023.06.05.
2023-06-05 00:00:00


AttributeError: 'str' object has no attribute 'strftime'

In [ ]:
for idx, ser in df2.iterrows():
    name = ser['name']
    blog_time = ser['time']
    print(name)
    cnt = 0

In [18]:
for idx, ser in df2.iterrows():
    name = ser['name']
    print(name)
    cnt = 0
    
    


    for blog_url in tqdm(ser['blog']):        
        # naver 블로그만 선택
        if blog_url.find('blog.naver.com') > 0:
                      
            driver = webdriver.Chrome()
            driver.get(blog_url)
            driver.switch_to.frame('mainFrame')

            try : 
                lis = driver.find_element(By.CLASS_NAME, 'se-main-container').find_elements(By.CLASS_NAME, 'se-text')
                blog_contents = []

                for li in lis:
                    content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())
                    blog_contents.append(content.replace('\U0001f979','').replace('\n',''))
                    text = ' '.join(blog_contents)

                cnt += 1
                accomo_contents.append(blog_contents)
                
            except : 
                try :
                    lis = driver.find_element(By.CLASS_NAME, 'postViewArea').find_elements(By.CLASS_NAME, 'se-text')
                    blog_contents = []

                    for li in lis:
                        content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())
                        blog_contents.append(content.replace('\U0001f979','').replace('\n',''))
                        text = ' '.join(blog_contents)

                    cnt += 1
                    accomo_contents.append(blog_contents)
                     
                except :
                     continue

            if cnt > 30:
                    break
    

100%|██████████| 1/1 [00:00<?, ?it/s]


In [19]:
accomo_contents

[]

In [ ]:
# # 스크롤 내리기
# for _ in range(10):
#     last_height = driver.execute_script("return document.body.scrollHeight")
#     for _ in range(5):
#         driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 
#         time.sleep(round(random.uniform(0.0, 1.2), 3))   
#     time.sleep(round(random.uniform(0.0, 1.2), 3))
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break

In [ ]:
lis = driver.find_element(By.CLASS_NAME, 'postViewArea').find_elements(By.CLASS_NAME, 'se-text')

In [26]:
url = 'https://blog.naver.com/youngsugi471/220403012299'
driver = webdriver.Chrome()
driver.get(url)




In [32]:
driver.find_element(By.CLASS_NAME, 'post-body')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".post-body"}
  (Session info: chrome=120.0.6099.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7F50B2142+3514994]
	(No symbol) [0x00007FF7F4CD0CE2]
	(No symbol) [0x00007FF7F4B776AA]
	(No symbol) [0x00007FF7F4BC1860]
	(No symbol) [0x00007FF7F4BC197C]
	(No symbol) [0x00007FF7F4C04EE7]
	(No symbol) [0x00007FF7F4BE602F]
	(No symbol) [0x00007FF7F4C028F6]
	(No symbol) [0x00007FF7F4BE5D93]
	(No symbol) [0x00007FF7F4BB4BDC]
	(No symbol) [0x00007FF7F4BB5C64]
	GetHandleVerifier [0x00007FF7F50DE16B+3695259]
	GetHandleVerifier [0x00007FF7F5136737+4057191]
	GetHandleVerifier [0x00007FF7F512E4E3+4023827]
	GetHandleVerifier [0x00007FF7F4E004F9+689705]
	(No symbol) [0x00007FF7F4CDC048]
	(No symbol) [0x00007FF7F4CD8044]
	(No symbol) [0x00007FF7F4CD8

8

In [24]:
url2 = 'https://blog.naver.com/imvaney/223121121422'
request2 = requests.get(url2)
soup2 = BeautifulSoup(request2.text, 'html.parser')

In [25]:
soup2


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ko">
<head>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="noindex,follow" name="robots"/>
<meta content="always" name="referrer"/>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="/favicon.ico?3" rel="shortcut icon" type="image/x-icon"/>
<link href="https://rss.blog.naver.com/imvaney.xml" rel="alternate" title="RSS feed for imvaney Blog" type="application/rss+xml"/>
<link href="https://blog.naver.com/NBlogWlwLayout.naver?blogId=imvaney" rel="wlwmanifest" type="application/wlwmanifest+xml"/>
<title>Vaney's Season Note : 네이버 블로그</title>
</head>
<script charset="UTF-8" src="https://ssl.pstatic.net/t.static.blog/mylog/versioning/Frameset-478668353_https.js" type="text/javascript"></script>
<script charset=